# Download time-series data from CMEMS

**Last updated: 29/04/2024**

**Copernicus Marine Environment Monitoring Service** (**CMEMS**, or Copernicus Marine Service for short) stores **L3**, **L4** and **reanalysis** satellite data from various satellite data providers. This script uses the conda environment `copernicusmarine`, the toolbox developed by Copernicus Marine Service, to download **time-series** data from the **Copernicus Marine Data Store**. I developed it with the assistance of the Copernicus Marine Service helpdesk officers.

This script is configured to download L3 and reanalysis time-series data for the following example locations:
* area of study around the **Endurance site** (**lat x lon**)
* locations of the **SmartBuoys** (**point locations**)

**You can use this script as a template and modify the code sections below to customise it for your own area of study.**

As of the latest update, certain CMEMS [products](https://marine.copernicus.eu/user-corner/user-notification-service/datasets-process-being-formatted-service-subset) (4km-global-CMEMSmulti and 300m-global-OLCI) are unavailable via the `copernicusmarine`, and have therefore been commented out from the product list.

## Import libraries and define paths

In [28]:
import os 
from pathlib import Path
import copernicusmarine
from datetime import datetime

**Modify the parameter names in the following window based on your needs.**

In [29]:
# Create the main download directories for our outputs
PATH_ROOT_DIR = Path.cwd().resolve().parents[1] # /absolute/path/to/two/levels/up
NAME_DOWNLOAD_DIR_AREASTUDY = 'data_timeseries_areastudy_CMT_nc'
NAME_DOWNLOAD_DIR_SMARTBUOY = 'data_timeseries_smartbuoy_CMT_nc'
SMARTBUOY_NAMES = ['Dowsing','NorthDogger','OysterGround']
full_path_download_dir_areastudy = os.path.join(PATH_ROOT_DIR,"data","raw","CMEMS_data",NAME_DOWNLOAD_DIR_AREASTUDY)
full_path_download_dir_smartbuoy = os.path.join(PATH_ROOT_DIR,"data","raw","CMEMS_data",NAME_DOWNLOAD_DIR_SMARTBUOY)
os.makedirs(full_path_download_dir_areastudy, exist_ok=True)
os.makedirs(full_path_download_dir_smartbuoy, exist_ok=True)

# Create subdirectories for SmartBuoy
smartbuoy_subdirectory_paths = [] # list to hold the full path names of created subdirectories
for i in range(len(SMARTBUOY_NAMES)):
    p = os.path.join(full_path_download_dir_smartbuoy, SMARTBUOY_NAMES[i])
    os.makedirs(p, exist_ok=True)
    smartbuoy_subdirectory_paths.append(p)

## Set the download parameters

**Modify as needed.**

In [30]:
# Define bounding box for our area of study

# Data won't be subset at those exact boundaries (too many digits)... 
#AREASTUDY_LONGITUDE_BOUNDS = [0.265, 1.801]
#AREASTUDY_LATITUDE_BOUNDS = [53.758, 54.656]

# ... thus expand the margins to make sure we encompass those lons and lats with the subsetting.
AREASTUDY_LONGITUDE_BOUNDS = [0.20, 1.89]
AREASTUDY_LATITUDE_BOUNDS = [53.70, 54.70]

In [31]:
# Define locations for the SmartBuoy systems

SMARTBUOY_LONGITUDE = [1.056,  2.280,  4.039]
SMARTBUOY_LATITUDE = [53.531, 55.683, 54.414]

In [32]:
# =====================================================
# List of datasets
# =====================================================

LIST_DATASET_IDS = [
 
# L3 satellite observations, global, various resolutions, daily, Copernicus-GlobColour algorithm
# Product ID: OCEANCOLOUR_GLO_BGC_L3_MY_009_103
    
    # 4 km resolution (multiple sensors merged)
    #"cmems_obs-oc_glo_bgc-plankton_my_l3-multi-4km_P1D", # Chlorophyll a + Diatoms + Dinoflagellates + Green algae + Haptophytes + Microphytoplankton + Nanophytoplankton + Picophytoplankton + Prochlorococcus + Prokaryotes (units: mg chla m-3) +  uncertainty (%) + flags
    #"cmems_obs-oc_glo_bgc-optics_my_l3-multi-4km_P1D",   # Particulate backscattering coeff (m-1) + Absorption coeff due to CDOM and non-algal particles (m-1) + uncertainty (%) + flags
    #"cmems_obs-oc_glo_bgc-transp_my_l3-multi-4km_P1D",   # Diffuse attenuation coeff kd490 (m-1) + SPM (g m-3) + Secchi disk depth (m) + uncertainty (%) + flags
    # 4 km resolution (OLCI sensor)
    "cmems_obs-oc_glo_bgc-plankton_my_l3-olci-4km_P1D",  # Chlorophyll a (mg chla m-3) + uncertainty (%) + flags
    "cmems_obs-oc_glo_bgc-transp_my_l3-olci-4km_P1D",    # Diffuse attenuation coeff kd490 (m-1) + uncertainty (%) + flags
    # 300 m resolution (OLCI sensor)
    #"cmems_obs-oc_glo_bgc-plankton_my_l3-olci-300m_P1D", # Chlorophyll a (mg chla m-3) + uncertainty (%) + flags
    
# L3 satellite observations, Atlantic-European NWS, various resolutions, daily
# Product ID: OCEANCOLOUR_ATL_BGC_L3_MY_009_113
    
    # 1 km resolution (multiple sensors merged)
    "cmems_obs-oc_atl_bgc-plankton_my_l3-multi-1km_P1D", # Chlorophyll a (mg chla m-3) + uncertainty (%) + flags
    "cmems_obs-oc_atl_bgc-optics_my_l3-multi-1km_P1D",   # Particulate backscattering coeff (m-1) + Absorption coeff due to CDOM and non-algal particles (m-1) + uncertainty (%) + flags
    "cmems_obs-oc_atl_bgc-transp_my_l3-multi-1km_P1D",   # Diffuse attenuation coeff kd490 (m-1) + SPM (g m-3) + Secchi disk depth (m) + uncertainty (%) + flags
    # 300 m resolution (OLCI sensor)
    "cmems_obs-oc_atl_bgc-plankton_my_l3-olci-300m_P1D", # Chlorophyll a (mg chla m-3) + uncertainty (%) + flags
      
# Biogeochemical reanalysis, Atlantic-European NWS, 7 km horizontal resolution, daily
# Product ID: NWSHELF_MULTIYEAR_BGC_004_011
    
    "cmems_mod_nws_bgc-chl_my_7km-3D_P1D-m",             # Chlorophyll a (mg chla m-3)
    "cmems_mod_nws_bgc-pft_my_7km-3D-diato_P1D-m",       # Diatoms (mg chla m-3)
    "cmems_mod_nws_bgc-pft_my_7km-3D-dino_P1D-m",        # Dinoflagellates (mg chla m-3)
    "cmems_mod_nws_bgc-pft_my_7km-3D-nano_P1D-m",        # Nanophytoplankton (mg chla m-3)
    "cmems_mod_nws_bgc-pft_my_7km-3D-pico_P1D-m",        # Picophytoplankton (mg chla m-3)
    "cmems_mod_nws_bgc-phyc_my_7km-3D_P1D-m",            # Phytoplankton concentration (mmol C m-3)
    "cmems_mod_nws_bgc-pp_my_7km-3D_P1D-m",              # Net primary production (mg C m-3 d-1)
    "cmems_mod_nws_bgc-kd_my_7km-3D_P1D-m",              # Attenuation coefficient kd (m-1)
    "cmems_mod_nws_bgc-no3_my_7km-3D_P1D-m",             # Nitrate (mmol m-3)
    "cmems_mod_nws_bgc-po4_my_7km-3D_P1D-m",             # Phosphate (mmol m-3)
    "cmems_mod_nws_bgc-o2_my_7km-3D_P1D-m",              # Oxygen (mmol m-3)
    "cmems_mod_nws_bgc-ph_my_7km-3D_P1D-m",              # pH
    "cmems_mod_nws_bgc-spco2_my_7km-2D_P1D-m",           # pCO2 (Pa)
    
# Physical reanalysis, Atlantic-European NWS, 7 km horizontal resolution
# Product ID: NWSHELF_MULTIYEAR_PHY_004_009
    "cmems_mod_nws_phy-mld_my_7km-2D_P1D-m",             # MLD (m), defined using sigma theta (daily, but there's also hourly)
    "cmems_mod_nws_phy-s_my_7km-3D_P1D-m",               # Salinity (PSU) (daily, but there's also hourly)
    "cmems_mod_nws_phy-t_my_7km-3D_P1D-m",               # Temperature (ºC) (daily, but there's also hourly)
    "cmems_mod_nws_phy-ssh_my_7km-2D_P1D-m",             # SSH (m) (daily, but there's also hourly)
    "cmems_mod_nws_phy-uv_my_7km-3D_P1D-m"               # Eastward + Northward seawater velocity (m s-1) (daily, but there's also hourly)
]

# ===============================================================================
# List of output file names (should correspond to the dataset names listed above)
# ===============================================================================

LIST_OUTPUT_NAMES = [
    #"obs_satell_glob_cmems_multi_4km_plk",
    #"obs_satell_glob_cmems_multi_4km_opt",
    #"obs_satell_glob_cmems_multi_4km_trns",
    "obs_satell_glob_cmems_olci_4km_plk",
    "obs_satell_glob_cmems_olci_4km_trns",
    #"obs_satell_glob_cmems_olci_300m_plk",
    "obs_satell_reg_cmems_multi_1km_plk",
    "obs_satell_reg_cmems_multi_1km_opt",
    "obs_satell_reg_cmems_multi_1km_trns",
    "obs_satell_reg_cmems_olci_300m_plk",
    "mod_bgc_reg_chl",
    "mod_bgc_reg_diat",
    "mod_bgc_reg_dino",
    "mod_bgc_reg_nano",
    "mod_bgc_reg_pico",
    "mod_bgc_reg_phy",
    "mod_bgc_reg_npp",
    "mod_bgc_reg_kd",
    "mod_bgc_reg_no3",
    "mod_bgc_reg_po4",
    "mod_bgc_reg_o2",
    "mod_bgc_reg_ph",
    "mod_bgc_reg_pco2",
    "mod_phy_reg_mld",
    "mod_phy_reg_sal",
    "mod_phy_reg_temp",
    "mod_phy_reg_ssh",
    "mod_phy_reg_velo"
]

## Exploratory analysis of one of the CMEMS datasets

In [33]:
# First, check available parameters in the toolbox copernicusmarine
help(copernicusmarine.subset)

Help on function subset in module copernicusmarine.python_interface.subset:

subset(dataset_url: Optional[str] = None, dataset_id: Optional[str] = None, dataset_version: Optional[str] = None, dataset_part: Optional[str] = None, username: Optional[str] = None, password: Optional[str] = None, variables: Optional[List[str]] = None, minimum_longitude: Optional[float] = None, maximum_longitude: Optional[float] = None, minimum_latitude: Optional[float] = None, maximum_latitude: Optional[float] = None, minimum_depth: Optional[float] = None, maximum_depth: Optional[float] = None, vertical_dimension_as_originally_produced: bool = True, start_datetime: Union[datetime.datetime, str, NoneType] = None, end_datetime: Union[datetime.datetime, str, NoneType] = None, subset_method: Literal['nearest', 'strict'] = 'nearest', output_filename: Optional[str] = None, file_format: Literal['netcdf', 'zarr'] = 'netcdf', service: Optional[str] = None, request_file: Union[pathlib.Path, str, NoneType] = None, outp

In [2]:
# Explore this dataset
DS = copernicusmarine.open_dataset(
    dataset_id = "cmems_obs-oc_glo_bgc-plankton_my_l3-olci-4km_P1D"
)
DS

NameError: name 'copernicusmarine' is not defined

## Downlaod data for our area of study

In [35]:
%%time

for dataset_id, output_name in zip(LIST_DATASET_IDS, LIST_OUTPUT_NAMES):
    
    print("Downloading dataset: ", dataset_id)

    copernicusmarine.subset(
        dataset_id = dataset_id,
        minimum_longitude = AREASTUDY_LONGITUDE_BOUNDS[0],
        maximum_longitude = AREASTUDY_LONGITUDE_BOUNDS[1],
        minimum_latitude = AREASTUDY_LATITUDE_BOUNDS[0],
        maximum_latitude = AREASTUDY_LATITUDE_BOUNDS[1],
        output_filename = f"{output_name}.nc",
        output_directory = f"{full_path_download_dir_areastudy}",
        force_download = True, # skips prompt "Do you want to proceed with download? [Y/n]:"
    )

print('All datasets downloaded!')

INFO - 2024-04-20T20:40:29Z - Dataset version was not specified, the latest one was selected: "202303"
INFO - 2024-04-20T20:40:29Z - Dataset part was not specified, the first one was selected: "default"
INFO - 2024-04-20T20:40:32Z - Service was not specified, the default one was selected: "arco-time-series"
INFO - 2024-04-20T20:40:34Z - Downloading using service arco-time-series...
INFO - 2024-04-20T20:40:35Z - Estimated size of the dataset file is 354.504 MB.
INFO - 2024-04-20T20:40:35Z - Writing to local storage. Please wait...


/Users/Anna/anaconda3/envs/copernicusmarine/lib/python3.11/site-packages/copernicusmarine/download_functions/download_arco_series.py:52: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  _filter_dimensions(preferred_chunks, dataset.dims.keys())


  0%|          | 0/2042 [00:00<?, ?it/s]

INFO - 2024-04-20T20:41:00Z - Successfully downloaded to /Users/Anna/LocalDocuments/Academic/Projects/Agile/matlab-jupyter-EBA-toolbox/data/raw/CMEMS_data/data_timeseries_areastudy_CMT_nc/obs_satell_glob_occci_multi_4km_plk.nc
INFO - 2024-04-20T20:41:02Z - Dataset version was not specified, the latest one was selected: "202207"
INFO - 2024-04-20T20:41:02Z - Dataset part was not specified, the first one was selected: "default"
INFO - 2024-04-20T20:41:03Z - Service was not specified, the default one was selected: "arco-time-series"
INFO - 2024-04-20T20:41:04Z - Downloading using service arco-time-series...
INFO - 2024-04-20T20:41:05Z - Estimated size of the dataset file is 31.988 MB.
INFO - 2024-04-20T20:41:05Z - Writing to local storage. Please wait...


/Users/Anna/anaconda3/envs/copernicusmarine/lib/python3.11/site-packages/copernicusmarine/download_functions/download_arco_series.py:52: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  _filter_dimensions(preferred_chunks, dataset.dims.keys())


...

  0%|          | 0/68 [00:00<?, ?it/s]

INFO - 2024-04-20T21:22:35Z - Successfully downloaded to /Users/Anna/LocalDocuments/Academic/Projects/Agile/matlab-jupyter-EBA-toolbox/data/raw/CMEMS_data/data_timeseries_areastudy_CMT_nc/mod_phy_reg_ssh.nc
INFO - 2024-04-20T21:22:36Z - Dataset version was not specified, the latest one was selected: "202012"
INFO - 2024-04-20T21:22:36Z - Dataset part was not specified, the first one was selected: "default"
INFO - 2024-04-20T21:22:37Z - Service was not specified, the default one was selected: "arco-time-series"
INFO - 2024-04-20T21:22:38Z - Downloading using service arco-time-series...
INFO - 2024-04-20T21:22:40Z - Estimated size of the dataset file is 984.916 MB.
INFO - 2024-04-20T21:22:40Z - Writing to local storage. Please wait...


/Users/Anna/anaconda3/envs/copernicusmarine/lib/python3.11/site-packages/copernicusmarine/download_functions/download_arco_series.py:52: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  _filter_dimensions(preferred_chunks, dataset.dims.keys())


  0%|          | 0/3170 [00:00<?, ?it/s]

INFO - 2024-04-20T21:22:56Z - Successfully downloaded to /Users/Anna/LocalDocuments/Academic/Projects/Agile/matlab-jupyter-EBA-toolbox/data/raw/CMEMS_data/data_timeseries_areastudy_CMT_nc/mod_phy_reg_velo.nc
All datasets downloaded!
CPU times: user 7min 1s, sys: 28min 46s, total: 35min 47s
Wall time: 42min 30s


## Download data for the SmartBuoy locations

In [34]:
%%time

for dataset_id, output_name in zip(LIST_DATASET_IDS, LIST_OUTPUT_NAMES):
    
    print("Downloading dataset: ", dataset_id)

    for i in range(len(SMARTBUOY_NAMES)):
        
        copernicusmarine.subset(
            dataset_id = dataset_id,
            minimum_longitude = SMARTBUOY_LONGITUDE[i],
            maximum_longitude = SMARTBUOY_LONGITUDE[i],
            minimum_latitude = SMARTBUOY_LATITUDE[i],
            maximum_latitude = SMARTBUOY_LATITUDE[i],
            output_filename = f"{output_name}.nc",
            output_directory = f"{smartbuoy_subdirectory_paths[i]}",
            force_download = True, # skips prompt "Do you want to proceed with download? [Y/n]:"
        )

print('All datasets downloaded!')

INFO - 2024-04-20T19:38:18Z - Dataset version was not specified, the latest one was selected: "202303"
INFO - 2024-04-20T19:38:18Z - Dataset part was not specified, the first one was selected: "default"
INFO - 2024-04-20T19:38:20Z - Service was not specified, the default one was selected: "arco-time-series"
INFO - 2024-04-20T19:38:21Z - Downloading using service arco-time-series...
INFO - 2024-04-20T19:38:23Z - Estimated size of the dataset file is 0.369 MB.
INFO - 2024-04-20T19:38:23Z - Writing to local storage. Please wait...


/Users/Anna/anaconda3/envs/copernicusmarine/lib/python3.11/site-packages/copernicusmarine/download_functions/download_arco_series.py:52: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  _filter_dimensions(preferred_chunks, dataset.dims.keys())


  0%|          | 0/242 [00:00<?, ?it/s]

INFO - 2024-04-20T19:38:31Z - Successfully downloaded to /Users/Anna/LocalDocuments/Academic/Projects/Agile/matlab-jupyter-EBA-toolbox/data/raw/CMEMS_data/data_timeseries_smartbuoy_CMT_nc/Dowsing/obs_satell_glob_occci_multi_4km_plk.nc
INFO - 2024-04-20T19:38:31Z - Dataset version was not specified, the latest one was selected: "202303"
INFO - 2024-04-20T19:38:31Z - Dataset part was not specified, the first one was selected: "default"
INFO - 2024-04-20T19:38:33Z - Service was not specified, the default one was selected: "arco-time-series"
INFO - 2024-04-20T19:38:34Z - Downloading using service arco-time-series...
INFO - 2024-04-20T19:38:35Z - Estimated size of the dataset file is 0.369 MB.
INFO - 2024-04-20T19:38:35Z - Writing to local storage. Please wait...


/Users/Anna/anaconda3/envs/copernicusmarine/lib/python3.11/site-packages/copernicusmarine/download_functions/download_arco_series.py:52: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  _filter_dimensions(preferred_chunks, dataset.dims.keys())


...

  0%|          | 0/1154 [00:00<?, ?it/s]

INFO - 2024-04-20T20:00:16Z - Successfully downloaded to /Users/Anna/LocalDocuments/Academic/Projects/Agile/matlab-jupyter-EBA-toolbox/data/raw/CMEMS_data/data_timeseries_smartbuoy_CMT_nc/WestGabbard/mod_phy_reg_velo.nc
INFO - 2024-04-20T20:00:16Z - Dataset version was not specified, the latest one was selected: "202012"
INFO - 2024-04-20T20:00:16Z - Dataset part was not specified, the first one was selected: "default"
INFO - 2024-04-20T20:00:17Z - Service was not specified, the default one was selected: "arco-time-series"
INFO - 2024-04-20T20:00:19Z - Downloading using service arco-time-series...
INFO - 2024-04-20T20:00:20Z - Estimated size of the dataset file is 4.104 MB.
INFO - 2024-04-20T20:00:20Z - Writing to local storage. Please wait...


/Users/Anna/anaconda3/envs/copernicusmarine/lib/python3.11/site-packages/copernicusmarine/download_functions/download_arco_series.py:52: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  _filter_dimensions(preferred_chunks, dataset.dims.keys())


  0%|          | 0/1154 [00:00<?, ?it/s]

INFO - 2024-04-20T20:00:24Z - Successfully downloaded to /Users/Anna/LocalDocuments/Academic/Projects/Agile/matlab-jupyter-EBA-toolbox/data/raw/CMEMS_data/data_timeseries_smartbuoy_CMT_nc/WestGabbard2/mod_phy_reg_velo.nc
All datasets downloaded!
CPU times: user 9min 41s, sys: 1min 17s, total: 10min 59s
Wall time: 22min 10s
